In [2]:
import os
import pandas as pd
from blast_utils import make_diamond_db, diamond_alignment

In [3]:
os.makedirs('ref_databases', exist_ok=True)
#make diamond reference database for BLAST
make_diamond_db('../../splits/task1/fastas/train_swissprot.fasta', 'ref_databases/train_swissprot') 

diamond v2.1.9.163 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: ../../splits/task1/fastas/train_swissprot.fasta
Opening the database file...  [0.015s]
Loading sequences...  [0.203s]
Masking sequences...  [0.07s]
Writing sequences...  [0.067s]
Hashing sequences...  [0.018s]
Loading sequences...  [0s]
Writing trailer...  [0.003s]
Closing the input file...  [0s]
Closing the database file...  [0.019s]

Database sequences  184529
  Database letters  72738365
     Database hash  e1cfdedf246106b48c94446badba64a5
        Total time  0.399000s


In [8]:
ref_withEC = pd.read_csv('../../processed_data/protein2EC.csv')
ref_withEC

,Entry,Entry Name,Sequence,EC number,Length,EC All,clusterRes30,clusterRes50,clusterRes70,clusterRes90,EC3,EC2,EC1
0,A0A009IHW8,ABTIR_ACIB9,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,3.2.2.6,269,3.2.2.6,A1AY86,A0A009IHW8,A0A009IHW8,A0A009IHW8,3.2.2,3.2,3
1,A0A023I7E1,ENG1_RHIMI,MRFQVIVAAATITMITSYIPGVASQSTSDGDDLFVPVSNFDPKSIF...,3.2.1.39,796,3.2.1.39,D4AZ24,A0A023I7E1,A0A023I7E1,A0A023I7E1,3.2.1,3.2,3
2,A0A024SC78,CUTI1_HYPJR,MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMP...,3.1.1.74,248,3.1.1.74,A8QPD8,A8QPD8,A8QPD8,A0A024SC78,3.1.1,3.1,3
3,A0A024SH76,GUX2_HYPJR,MIVGILTTLATLATLAASVPLEERQACSSVWGQCGGQNWSGPTCCA...,3.2.1.91,471,3.2.1.91,B2AE04,A1CCN4,A0A024SH76,A0A024SH76,3.2.1,3.2,3
4,A0A044RE18,BLI_ONCVO,MYWQLVRILVLFDCLQKILAIEHDSICIADVDDACPEPSHTVMRLR...,3.4.21.75,693,3.4.21.75,Q9VBC7,A0A044RE18,A0A044RE18,A0A044RE18,3.4.21,3.4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185990,Q05115,AMDA_BORBO,MQQASTPTIGMIVPPAAGLVPADGARLYPDLPFIASGLGLGSVTPE...,4.1.1.76,240,4.1.1.76,Q05115,Q05115,Q05115,Q05115,4.1.1,4.1,4
185991,Q6HX62,Y3032_BACAN,MGQNQFRWSNEQLREHVEIIDGTRSPHKLLKNATYLNSYIREWMQA...,3.5.4.2,584,3.5.4.2,Q9KF49,Q6HX62,Q6HX62,Q6HX62,3.5.4,3.5,3
185992,Q6L032,Y1085_PICTO,MLLKNIKISNDYNIFMIIASRKPSLKDIYKIIKVSKFDEPADLIIE...,3.5.4.2,573,3.5.4.2,Q9KF49,Q6L032,Q6L032,Q6L032,3.5.4,3.5,3
185993,Q94MV8,VG56_BPLZ5,MAHFNECAHLIEGVDKANRAYAENIMHNIDPLQVMLDMQRHLQIRL...,3.6.1.12,172,3.6.1.12,P39262,P39262,Q94MV8,Q94MV8,3.6.1,3.6,3


In [10]:
entry2EC = dict(zip(ref_withEC['Entry'], ref_withEC['EC All']))

In [13]:
names= ['price_protein_test', 'promiscuous_protein_test', '30_protein_test', '30-50_protein_test']

for name in names:
    #plt.subplot(2, 3, i+1)
    df = pd.read_csv(f'../../splits/task1/{name}.csv')
    results = diamond_alignment(f'../../splits/task1/fastas/{name}.fasta', 'ref_databases/train_swissprot')
    #turn indices into a column
    results['Entry'] = results.index
    #merge the two dataframes
    merged = pd.merge(df, results, on='Entry')
    #add the EC numbers
    merged['EC Predicted'] = merged['Entry'].map(entry2EC)

In [14]:
merged

,Entry,Entry Name,Sequence,EC number,Length,EC All,clusterRes30,clusterRes50,clusterRes70,clusterRes90,EC3,EC2,EC1,aln_coverage,max_id,ref_entry_id,EC Predicted
0,Q0KBD2,DEND_CUPNH,MNVLITGGAGFLGLQLARLLLQRGTLNLDGQPVAIKRLTLLDVVAP...,1.1.1.410,324,1.1.1.410,P44094,Q0KBD2,Q0KBD2,Q0KBD2,1.1.1,1.1,1,99.0,46.5300,P44094,1.1.1.410
1,D4GYH5,AGLM_HALVD,MELSIIGSGYVGTTIAACFAELGHDVVNVDIDEDIVASLNDGQAPI...,1.1.1.22,430,1.1.1.22,O34862,D4GYH5,D4GYH5,D4GYH5,1.1.1,1.1,1,99.1,42.4148,O54068,1.1.1.22
2,P9WQC7,ADHB_MYCTU,MKTKGALIWEFNQPWSVEEIEIGDPRKDEVKIQMEAAGMCRSDHHL...,1.1.1.1,375,1.1.1.1,P80094,P9WQC7,P9WQC7,P9WQC7,1.1.1,1.1,1,100.0,46.9000,P80175,1.1.1.1
3,Q94AX4,DLD_ARATH,MAFASKFARSKTILSFLRPCRQLHSTPKSTGDVTVLSPVKGRRRLP...,1.1.2.4,567,1.1.2.4,Q12627,Q94AX4,Q94AX4,Q94AX4,1.1.2,1.1,1,86.7,40.0554,F1QXM5,1.1.2.4
4,B8MKR3,CYB2_TALSN,MARVLDAAEVAKHNTPESCWVILYGKVYDVTEFISSHPGGVKVILR...,1.1.2.3,496,1.1.2.3,B8MKR3,B8MKR3,B8MKR3,B8MKR3,1.1.2,1.1,1,84.1,31.8739,P00175,1.1.2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Q9WXX0,RBSA1_THEMA,MFPLLAFRGDRMEILKAKGIVKRFPGVVAVDNVDFEVYENEIVSLI...,7.5.2.7,520,7.5.2.7,A4WER4,Q9WXX0,Q9WXX0,Q9WXX0,7.5.2,7.5,7,97.8,48.0198,Q891M1,7.5.2.7
522,Q831L8,TAGH_ENTFA,MEKELKVRTKLLTKEYSLAQTRIDKLKTLFSVFQNKVPTFWALKGV...,7.5.2.4,447,7.5.2.4,Q03SI5,Q831L8,Q831L8,Q831L8,7.5.2,7.5,7,95.7,39.8112,Q9CH26,7.5.2.4
523,Q9C8T1,AB1I_ARATH,MSIRRPQIPRLLLQNVSCMRNAQQILRHVNVSLHDGGALVLTGTNG...,7.6.2.5,229,7.6.2.5,Q2G9A9,Q9C8T1,Q9C8T1,Q9C8T1,7.6.2,7.6,7,91.7,36.4966,Q2VZJ1,7.6.2.5
524,Q0A808,CCMA_ALKEH,MGDTALHLVAAPALEATGLQVARGGRPLFRGLGFRLARGGLLCVRG...,7.6.2.5,236,7.6.2.5,Q2G9A9,Q0A808,Q0A808,Q0A808,7.6.2,7.6,7,86.7,39.6219,Q46RX0,7.6.2.5
